In [4]:
import sys
from pathlib import Path

# assume this notebook lives in notebooks/, so parent() is the repo root
sys.path.append(str(Path().resolve().parent))
from paths import PROJECT_ROOT

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path

# DreaMS imports
from dreams.utils.dformats import DataFormatA
from dreams.utils.data     import MSData
from dreams.utils.io       import append_to_stem

/Users/macbook/UTILS/anaconda3/envs/dreams_mimb/lib/python3.11/site-packages/lightning_fabric/__init__.py:41: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
Determination of memory status is not supported on this 
 platform, measuring for memoryleaks will never fail


In [7]:
# 2) point to your file (mzML or HDF5)
mzml_path = PROJECT_ROOT / "data" / "rawfiles" / "202312_20_P09-Leaf-r1_1uL.mzML"
h5_path   = mzml_path.with_suffix(".hdf5")

In [8]:
# 3) load (or reuse cache) into an MSData
if h5_path.exists():
    msdata = MSData.from_hdf5(str(h5_path), in_mem=True)
else:
    msdata = MSData.from_mzml(str(mzml_path))


Loading dataset 202312_20_P09-Leaf-r1_1uL into memory (5681 spectra)...


In [9]:
# 4) pull out spectra & precursor m/z arrays
spectra  = msdata.get_spectra()      # (N, 2, P)
prec_mzs  = msdata.get_prec_mzs()    # (N,)


In [14]:
spectra.shape

(5681, 2, 128)

In [15]:
# 5) compute single‐spectrum quality flags
dformat      = DataFormatA()
quality_lvls = [
    dformat.val_spec(spec, prec, return_problems=True)
    for spec, prec in zip(spectra, prec_mzs)
]

/Users/macbook/UTILS/anaconda3/envs/dreams_mimb/lib/python3.11/site-packages/dreams/utils/spectra.py:258: RuntimeWarning: divide by zero encountered in scalar divide
  return max(peak_list[1]) / min(peak_list[1])


In [16]:
# 6) tally pass/fail
qc_counts = pd.Series(quality_lvls).value_counts()
print("QC breakdown:\n", qc_counts)

QC breakdown:
 All checks passed                      4705
Number of high intensity peaks >= 3     938
Intensity amplitude >= 20.0              23
m/z range <= 1000.0                      15
Name: count, dtype: int64


In [17]:
# 7) select only 'All checks passed'
keep_idx = np.where(np.array(quality_lvls) == "All checks passed")[0]

In [18]:
# 8) write high‐quality subset to new HDF5
hq_h5 = append_to_stem(h5_path, "high_quality")
msdata.form_subset(idx=keep_idx, out_pth=hq_h5)
print("Wrote high-quality file to:", hq_h5)

Wrote high-quality file to: /Users/macbook/CODE/DreaMS_MIMB/data/rawfiles/202312_20_P09-Leaf-r1_1uL_high_quality.hdf5


In [19]:
# 9) verify by loading it back
msdata_hq = MSData.load(hq_h5)
print("High-quality MSData:", msdata_hq)

High-quality MSData: MSData(pth=/Users/macbook/CODE/DreaMS_MIMB/data/rawfiles/202312_20_P09-Leaf-r1_1uL_high_quality.hdf5, in_mem=False) with 4,705 spectra.
